<a href="https://colab.research.google.com/github/Divyam-Deep/Guess-the-word-/blob/main/Guess_the_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import gensim.downloader as api
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import random
import time

In [39]:
# Function to load Word2Vec model with retries
def load_word2vec_model(model_name, retries=3, wait_time=5):
    for attempt in range(retries):
        try:
            wv = api.load(model_name)
            return wv
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
    # Fallback to a smaller model if retries are exhausted
    print("Falling back to a smaller model: glove-wiki-gigaword-50")
    return api.load('glove-wiki-gigaword-50')


In [40]:
# Load pre-trained Word2Vec model
wv = load_word2vec_model('word2vec-google-news-300')


In [41]:
# Predefined list of common words (for example, this can be expanded or refined)
common_words = [
    'time', 'person', 'year', 'way', 'day', 'thing', 'man', 'world', 'life', 'hand',
    'part', 'child', 'eye', 'woman', 'place', 'work', 'week', 'case', 'point', 'government',
    'company', 'number', 'group', 'problem', 'fact', 'apple', 'banana', 'orange', 'grape',
    'breakfast', 'morning', 'evening', 'computer', 'science', 'music', 'school', 'teacher',
    'student', 'friend', 'family', 'mother', 'father', 'brother', 'sister', 'house', 'car',
    'city', 'country', 'water', 'food', 'dog', 'cat', 'mouse', 'bird', 'fish', 'tree', 'flower'
]

In [42]:
# Filter words to ensure they are in the Word2Vec model
filtered_common_words = [word for word in common_words if word in wv]


In [43]:
# Function to generate a random word
def generate_random_word():
    return np.random.choice(filtered_common_words)


In [44]:
# Function to compute similarity
def compute_similarity(word1, word2):
    if word1 in wv and word2 in wv:
        vec1 = wv[word1].reshape(1, -1)
        vec2 = wv[word2].reshape(1, -1)
        similarity = cosine_similarity(vec1, vec2)
        return similarity[0][0]
    else:
        return 0.0

In [55]:
# Function to find similar words
def find_similar_words(word, topn=10):
    if word in wv:
        similar_words = wv.most_similar(word, topn=topn)
        # Filter similar words to keep only those in the common words list
        filtered_similar_words = [sim_word for sim_word, _ in similar_words if sim_word in filtered_common_words]
        # If not enough similar words, use any available similar words from the model
        if len(filtered_similar_words) < 3:
            filtered_similar_words = [sim_word for sim_word, _ in similar_words]
        return filtered_similar_words[:1]
    else:
        return []


In [56]:
# Function to create a hint sentence
def create_hint_sentence(similar_words):
    hints = ", ".join(similar_words[:3])
    return f"A word similar to this: {hints}"

In [57]:
# Function to display similarity as a bar
def display_similarity(attempts, similarities):
    print("\nGuesses and their similarities:")
    for attempt, sim in zip(attempts, similarities):
        bar = '█' * int(sim * 50)
        print(f"{attempt}: {sim:.2f} {bar}")

In [63]:
# def run_game_timer():
#     start_time = time.time()
#     while time.time() - start_time < 60:
#         remaining_time = int(60 - (time.time() - start_time))
#         clear_output(wait=True)
#         print(f"Time left: {remaining_time} seconds")
#         user_input = input("\nEnter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ").strip().lower()
#         if user_input in ["new", "n", "ans", "answer"]:
#             return user_input, True
#     return None, False

In [68]:
# Game logic
def word_guessing_game():
    while True:
        generated_word = generate_random_word()
        similar_words = find_similar_words(generated_word)
        hint_sentence = create_hint_sentence(similar_words)

        print(f"\nNew game started. Hint: The word has {len(generated_word)} letters.")
        print(hint_sentence)

        attempts = []
        similarities = []

        while True:
            user_input = input("\nEnter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ").strip().lower()
            if user_input in ["new", "n"]:
                break
            elif user_input in ["ans", "answer"]:
                print(f"The word was: {generated_word}")
                break
            elif user_input == generated_word:
                print("Congratulations! You've guessed the correct word!")
                break

            similarity = compute_similarity(user_input, generated_word)
            attempts.append(user_input)
            similarities.append(similarity)

            clear_output(wait=True)
            print(f"Hint: The word has {len(generated_word)} letters.")
            print(hint_sentence)
            display_similarity(attempts, similarities)

        if user_input in ["new", "n", "ans", "answer"]:
            continue

In [69]:
# Run the game
word_guessing_game()

Hint: The word has 6 letters.
A word similar to this: someone

Guesses and their similarities:
random: 0.22 ███████████

Enter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ans
The word was: person

New game started. Hint: The word has 4 letters.
A word similar to this: integral_part

Enter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ans
The word was: part

New game started. Hint: The word has 4 letters.
A word similar to this: day

Enter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): ans
The word was: time

New game started. Hint: The word has 7 letters.
A word similar to this: afternoon

Enter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): new

New game started. Hint: The word has 6 letters.
A word similar to this: elementary

Enter your guess (or type 'new' to start a new game, 'ans' or 'answer' to reveal the word): new

New 

KeyboardInterrupt: Interrupted by user